In [6]:
import mne
import matplotlib.pyplot as plt
import pre_process as pp
import utils as u
import pandas as pd
import time
import joblib as jb
from tqdm import tqdm
import os

In [7]:
path_list = u.get_path_list('D:/isip.piconepress.com/projects/nedc/data/tuh_eeg/tuh_eeg_abnormal/v3.0.1/edf',['.edf'],True)

In [3]:
print(type(path_list))
print(path_list.shape)

<class 'numpy.ndarray'>
(2993,)


In [4]:
path_id_list= [f"campione_{i}" for i in range(1, 2994)]

In [5]:
df_path = pd.DataFrame({'Path_id':path_id_list, 'Path':path_list})

In [6]:
df_path.tail()

,Path_id,Path
2988,campione_2989,D:\isip.piconepress.com\projects\nedc\data\tuh...
2989,campione_2990,D:\isip.piconepress.com\projects\nedc\data\tuh...
2990,campione_2991,D:\isip.piconepress.com\projects\nedc\data\tuh...
2991,campione_2992,D:\isip.piconepress.com\projects\nedc\data\tuh...
2992,campione_2993,D:\isip.piconepress.com\projects\nedc\data\tuh...


In [7]:
df_path.to_csv('path.csv', index=False, encoding='utf-8-sig')

In [ ]:
def worker(in_file, out_dir, out_prefix):
    try:
        dg = pp.data_gen(in_file, out_dir, out_prefix)
        dg.save_final_data(seg_len=1.0, merge_len=0.2)
    except:
        flag = False
        time.sleep(3.0)
    else:
        flag = True
    return in_file, flag

In [3]:
md_file = "path.csv"
out_dir = "D:/raw_data"
log_file = "D:/raw_data/log.csv"

In [4]:
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

df_path = pd.read_csv(md_file)
tasks = [(Pt, Pt_id) for Pt, Pt_id in zip(df_path.Path, df_path.Path_id)]
tasks[0]

('D:\\isip.piconepress.com\\projects\\nedc\\data\\tuh_eeg\\tuh_eeg_abnormal\\v3.0.1\\edf\\eval\\abnormal\\01_tcp_ar\\aaaaabdo_s003_t000.edf',
 'campione_1')

In [5]:
n_jobs = jb.cpu_count()
res = jb.Parallel(n_jobs=1)(jb.delayed(worker)(in_file, out_dir, out_f_id) for in_file, out_f_id in tqdm(tasks))

  0%|          | 0/2993 [00:00<?, ?it/s]

19
<class 'list'>


  0%|          | 1/2993 [00:02<2:06:33,  2.54s/it]

19
<class 'list'>


  0%|          | 2/2993 [00:05<2:04:39,  2.50s/it]

...

19
<class 'list'>


  0%|          | 7/2993 [00:18<2:21:48,  2.85s/it]

19
<class 'list'>


  0%|          | 8/2993 [00:21<2:23:12,  2.88s/it]

KeyboardInterrupt: 

In [ ]:
df_log = pd.DataFrame(res, columns=['path', 'status'])
df_log.to_csv(log_file, index=False, encoding="utf-8-sig") 

In [7]:
g = [(0, True), (1, True), (2, True)]

In [8]:
list(zip(*g))

[(0, 1, 2), (True, True, True)]